In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [37]:
filepath = "C:/Users/laksh/OneDrive/Desktop/Sura/dataset_updated.csv"
data = pd.read_csv(filepath)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data.head()

,Shear Rate,Viscosity,Particle Size,Temperature,Temperature-(T')
0,0.100,160000.0,100,20,19.93
1,0.120,138000.0,100,20,19.93
2,0.143,119000.0,100,20,19.93
3,0.172,103000.0,100,20,19.93
4,0.205,89444.0,100,20,19.94


In [38]:
from sklearn.model_selection import train_test_split
X = data[['Particle Size','Temperature','Shear Rate']]
y = data['Viscosity']
X_train,X_test,y_train,y_test = train_test_split(X,y)
X_test

,Particle Size,Temperature,Shear Rate
489,500,40,161.000
294,500,30,1.240
352,100,40,1.780
144,500,20,31.800
447,300,40,2000.000
...,...,...,...
123,500,20,0.725
36,100,20,65.300
163,500,20,973.000
98,300,20,192.000


In [39]:
from sklearn.tree import DecisionTreeRegressor
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 123) 

# xgb_r.fit(X_train, y_train) 
  
# # Predict the model 
# pred = xgb_r.predict(X_test) 
  
# # RMSE Computation 
# # rmse = np.sqrt(MSE(y_test, pred)) 
# # print("RMSE : % f" %(rmse)) 
# mean_absolute_percentage_error(y_test,pred)
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_train,y_train)

DecisionTreeRegressor(random_state=0)

In [40]:
regressor.score(X_test,y_test)
y_pred = regressor.predict(X_test)
df = pd.DataFrame({
    'Actual': y_test,
    'DecisionTree_Predicted': y_pred,
    # 'XGB_Predicted': xgb_pred
})

# # print(df)
# single_test_case = np.array([230, 20, 100])

# # Reshape the single test case to a 2D array
# single_test_case = single_test_case.reshape(1, -1)

# # Predict using the DecisionTreeRegressor
# dt_pred_single = regressor.predict(single_test_case)
# print(dt_pred_single)


In [41]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
mean_absolute_percentage_error(y_test,y_pred)

0.320171511932991

In [18]:
import tensorflow as tf

# Verify the shape of X_train
print("X_train shape:", X_train.shape)

# Define normalization layer and adapt it to the training data
xyz = tf.concat(X_train, 1)
norm_layer = tf.keras.layers.Normalization(input_shape=(3,), axis=-1)
norm_layer.adapt(xyz)

# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),  # Ensure the input shape matches the number of features in X_train
    norm_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear'),
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=200, batch_size=10, verbose=1)

# Evaluate the model
mse_test, rmse_test = model.evaluate(X_test, y_test)

print("Test MSE:", mse_test)
print("Test RMSE:", rmse_test)
y_pred = model.predict(X_test)
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
mean_absolute_percentage_error(y_test,y_pred)


X_train shape: (378, 3)


c:\Users\laksh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 603829760.0000 - root_mean_squared_error: 24001.2676 - val_loss: 1013268160.0000 - val_root_mean_squared_error: 31831.8750
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 657729536.0000 - root_mean_squared_error: 25439.7871 - val_loss: 950326720.0000 - val_root_mean_squared_error: 30827.3691
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 898964480.0000 - root_mean_squared_error: 29772.0410 - val_loss: 799570304.0000 - val_root_mean_squared_error: 28276.6738
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 651324672.0000 - root_mean_squared_error: 25374.3438 - val_loss: 724711104.0000 - val_root_mean_squared_error: 26920.4590
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 309113056.0000 - root_mean_squared_error: 17310.2578 - val_loss: 715730368.0000 - val_root_mean_squared_error: 26753.1367
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 523198400.0000 - root_mean_squared_

173.82421605115488

things to read for imporving it - 
1. what process to use if data is not continuous and discrete like 100,300,500
2. what changes to hyperparameters can be done
3. remove outliers from the dataset
4. any other method to curve fit